In [3]:
import pandas as pd
from utils import aggregate_expert_opinions
metric_results = {}
llm_type = 'gpt'
num_seed = 1
for dataset_name in ['en', 'eu', 'jp']:
    gold = pd.read_csv(f'../step5-close_llm_baseline/data/{dataset_name}.csv')[['job_title', 'job_description', 'task', 'bert_task']]
    gold['bert_task'] = gold['bert_task'].apply(eval)
    for seed in range(num_seed):
        ans = []
        for turn in range(8):
            df = pd.read_csv(f'../step5-close_llm_baseline/results/{dataset_name}/{llm_type}/seed{seed}/result{turn}.csv') 
            ans.append(df)
        ans = pd.concat(ans)
        ans.columns = ['job_title', 'job_description', f'{llm_type}_task_seed{seed}']
        ans[f'{llm_type}_task_seed{seed}'] = ans[f'{llm_type}_task_seed{seed}'].apply(eval)
        gold = pd.merge(gold, ans, on=['job_title', 'job_description'])
    gold['gpt_task'] = gold.apply(lambda x: aggregate_expert_opinions([x[f'{llm_type}_task_seed{i}'] for i in range(num_seed)], top_num=10)[0], axis=1)
    gold = gold[['job_title', 'job_description', 'task', f'{llm_type}_task', 'bert_task']]
    gold.to_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv', index=None)
    gold.to_excel(f'results/{dataset_name}/{dataset_name}-{llm_type}.xlsx', index=None)

In [43]:
for dataset_name in ['eu']:
    gold = pd.read_excel(f'results/{dataset_name}/{dataset_name}-{llm_type}-annotated.xlsx')
    gold['bert_task'] = gold['bert_task'].apply(eval)
    gold['gpt_task'] = gold['gpt_task'].apply(eval)
    gold = gold.fillna(0)
    gold['task'] = gold.apply(lambda x: x['gpt_task'][0] if x['annotated'] == 1 else x['task'], axis=1)
    gold = gold.drop('annotated', axis=1)
    gold.to_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv', index=None)

In [4]:
import pandas as pd
metric_results = {}
llm_type = 'gpt'
for dataset_name in ['en', 'eu', 'jp']:
    metric_results[dataset_name] = {}
    ans = pd.read_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv')
    ans[f'{llm_type}_task'] = ans[f'{llm_type}_task'].apply(eval)
    for k in [5, 10]:
        hits = ans.apply(lambda x: x['task'] in x[f'{llm_type}_task'][:k], axis=1).sum() / len(ans)
        metric_results[dataset_name][f"hits@{k}"] = hits
pd.DataFrame(metric_results)

,en,eu,jp
hits@5,0.654135,0.44,0.203883
hits@10,0.684211,0.52,0.349515


In [48]:
import pandas as pd
metric_results = {}
for dataset_name in ['en', 'eu', 'jp']:
    metric_results[dataset_name] = {}
    gold = pd.read_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv')
    gold['bert_task'] = gold['bert_task'].apply(eval)
    for k in [5, 10]:
        hits_10 = gold.apply(lambda x: x['task'] in x[f'bert_task'][:k], axis=1).sum() / len(gold)
        metric_results[dataset_name][f"hits@{k}"] = hits_10
pd.DataFrame(metric_results)

,en,eu,jp
hits@5,0.255639,0.16,0.147059
hits@10,0.315789,0.24,0.205882


In [1]:
import pandas as pd
metric_results = {}
llm_type = 'gpt'
for dataset_name in ['en']:
    metric_results[dataset_name] = {}
    gold_white = pd.read_csv(f'../step3-bert_recall/CuBERT/results/{dataset_name}/2e-05_32_0.8/{dataset_name}-white.csv', sep='\t')
    # gold_white = pd.read_csv(f'/data/chenxi/research/conference/TMIS/COTR/step3-bert_recall/CuBERT/results/en/5e-05_32_0.8-white/en-white.csv', sep='\t')
    gold = pd.read_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv').drop('bert_task', axis=1)
    gold_white['bert_task'] = gold_white.apply(lambda x: [x[f'task{i}'] for i in range(100)], axis=1)
    gold_white = gold_white[['job_title', 'job_description', 'bert_task']]
    gold = pd.merge(gold, gold_white)
    for k in [5, 10, 100]:
        hits_10 = gold.apply(lambda x: x['task'] in x[f'bert_task'][:k], axis=1).sum() / len(gold)
        metric_results[dataset_name][f"hits@{k}"] = hits_10
pd.DataFrame(metric_results)

,en
hits@5,0.000000
hits@10,0.007519
hits@100,0.015038


In [31]:
pd.read_csv(f'results/{dataset_name}/{dataset_name}-{llm_type}.csv').drop('bert_task', axis=1)

,job_title,job_description,task,gpt_task
0,"VP, Sr Retail Bus Banker - Illinois Market","Spend majority of time in the field, on sales ...",Contact current or potential customers to prom...,['Develop proposals for current or prospective...
1,Associate Account Executive,Assist in executing marketing client campaigns...,Contact current or potential customers to prom...,['Develop proposals for current or prospective...
2,Aftermarket Sales Representative,Develop a territory management plan to maximiz...,Contact current or potential customers to prom...,['Coordinate project activities with other per...
3,Commercial Lines Producer,"Prospect new business primarily by phone, mail...",Contact current or potential customers to prom...,['Contact current or potential customers to pr...
4,Regional Sales Development Manager,Effectively communicate voice of customer (VOC...,Contact current or potential customers to prom...,['Recommend products or services to customers....
...,...,...,...,...
128,IT Project Manager,- Coordinate with technical leads and develope...,Confer with technical personnel to prepare des...,['Develop computer or information security pol...
129,Public Relations Manager - Networking,Support content development to promote Cisco s...,Write informational material.,"['Develop promotional strategies or plans.', '..."
130,Program Security Officer-Journeyman,Develop security plans and procedures for safe...,"Develop safety standards, policies, or procedu...",['Develop computer or information security pol...
131,"Director of Finance - The Langham, New York",Direct accounting staff to ensure accurate acc...,"Direct sales, marketing, or customer service a...",['Explain financial information to customers.'...


In [20]:
0.030075*133

3.999975

In [23]:
import pandas as pd
df = pd.read_csv('data/en.csv')
df

,job_title,job_description,task,gpt_task,llm_task,bert_task
0,"VP, Sr Retail Bus Banker - Illinois Market","Spend majority of time in the field, on sales ...",Contact current or potential customers to prom...,['Contact current or potential customers to pr...,['Contact current or potential customers to pr...,['Evaluate personnel practices to ensure adher...
1,Associate Account Executive,Assist in executing marketing client campaigns...,Contact current or potential customers to prom...,['Contact current or potential customers to pr...,['Contact current or potential customers to pr...,['Evaluate personnel practices to ensure adher...
2,Aftermarket Sales Representative,Develop a territory management plan to maximiz...,Contact current or potential customers to prom...,['Contact current or potential customers to pr...,['Contact current or potential customers to pr...,['Prepare informational or reference materials...
3,Commercial Lines Producer,"Prospect new business primarily by phone, mail...",Contact current or potential customers to prom...,['Contact current or potential customers to pr...,['Contact current or potential customers to pr...,['Evaluate personnel practices to ensure adher...
4,Regional Sales Development Manager,Effectively communicate voice of customer (VOC...,Contact current or potential customers to prom...,['Contact current or potential customers to pr...,['Contact current or potential customers to pr...,['Prepare informational or reference materials...
...,...,...,...,...,...,...
129,IT Project Manager,- Coordinate with technical leads and develope...,Confer with technical personnel to prepare des...,['Confer with technical personnel to prepare d...,['Develop computer or information security pol...,['Apply information technology to solve busine...
130,Public Relations Manager - Networking,Support content development to promote Cisco s...,Write informational material.,"['Write informational material.', 'Collaborate...","['Write advertising or promotional material.',...","['Manage guest services.', 'Negotiate for serv..."
131,Program Security Officer-Journeyman,Develop security plans and procedures for safe...,"Develop safety standards, policies, or procedu...","['Develop safety standards, policies, or proce...",['Develop computer or information security pol...,['Provide recommendations to others about comp...
132,"Director of Finance - The Langham, New York",Direct accounting staff to ensure accurate acc...,"Direct sales, marketing, or customer service a...","['Direct sales, marketing, or customer service...","['Perform marketing activities.', 'Negotiate f...",['Explain financial information to customers.'...


In [28]:
df['task'].unique()

array(['Contact current or potential customers to promote products or services.',
       'Develop business or market strategies.',
       'Develop marketing plans or strategies.',
       'Communicate with customers to resolve complaints or ensure satisfaction.',
       'Prepare financial documents, reports, or budgets.',
       'Develop promotional strategies or plans.',
       'Develop computer or information security policies or procedures.',
       'Liaise between departments or other groups to improve function or communication.',
       'Evaluate applicable laws and regulations to determine impact on organizational activities.',
       'Develop organizational policies or programs.',
       'Collaborate with others to develop or implement marketing strategies.',
       'Write advertising or promotional material.',
       'Develop procedures for data management.',
       'Market products, services, or events.',
       'Develop financial or business plans.',
       'Develop content fo

In [9]:
0.2556*133

33.9948

In [10]:
0.3157*133

41.988099999999996